In [2]:
import os
import re
import string
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np

### Dynamic few-shot prompts
- this next last
- Monday Tuesday Wendesday Thursday Friday Saturday Sunday week weekend weekday
- morning afternoon evening night

### Fuzzy keywords matching
 - meeting search via topic
 - eamil name and attendee names matching 


Question: What is my **latest** scheduled meeting for today? tell me meeting ID
Question: Do I have **database lecture** today, true or false?   
Question: How many meetings did I attended **last** week? 
Question: How many meetings do I have this tuesday?
Question: How many meetings do I have tuesday morning this week?
Question: Do I have meeting this tuesday afternoon, True or False??
Question: How many meetings do I have in this tuesday afternoon?
Question: How many meetings do I have this weekend?
Question: How many meetings do I have this week in total?
Question: Do I have meeting next weekend, True or False?
Question: How many meetings do I have next weekend?
Question: How many meetings do I have this weekday?
Question: How many meetings which are yet to start this week?

Question: Find the meeting I attended with salesteam, tell me the meeting ID?
Question: How many meetings do I have meeting in this afternoon?



# Error analysis
1. 

In [3]:
today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
print(type(today_date))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Question: How many attendees are there for the meeting with ID 'masbk72a24cb0a8k9c7jo0e9s6'?
 True answer: 2
 Wrong answer: 1
 ```python
 def query_dataframe(calendar_data):
    
    # Filter the dataframe to get the specific meeting by ID
    specific_meeting = calendar_data[calendar_data['ID'] == 'masbk72a24cb0a8k9c7jo0e9s6']
    
    # Get the attendees for the specific meeting
    if not specific_meeting.empty:
        attendees_count = specific_meeting['attendees'].str.split('\n').apply(len).sum()
    else:
        attendees_count = 0
        
    return attendees_count
 ```

## Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

```python
def query_dataframe(calendar_data):
    # get today's date  ---   <class 'pandas._libs.tslibs.timestamps.Timestamp'>
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # Filter the dataframe for this week
    start_of_week = today_date - pd.DateOffset(days=today_date.dayofweek)

    end_of_week = start_of_week + pd.DateOffset(days=6)

    this_week_data = calendar_data[(pd.to_datetime(calendar_data['start']) >= start_of_week) & (pd.to_datetime(calendar_data['end']) <= end_of_week)]
    
    # Filter for Tuesday mornings
    tuesday_morning_data = this_week_data[(pd.to_datetime(this_week_data['start']).dt.dayofweek == 1) & (pd.to_datetime(this_week_data['start']).dt.hour < 12)]
    
    # Count the number of meetings
    num_meetings = len(tuesday_morning_data)
    
    return num_meetings
```

## .dt accessor with datetimelike values

### 1
Question: How many events do I have scheduled for today?
 True answer: 2
 Non-executable code: Can only use .dt accessor with datetimelike values
 ```python
import pandas as pd

def query_dataframe(calendar_data):
    
    # get today's date
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # Filter the dataframe to include only events scheduled for today
    today_events = calendar_data[(calendar_data['start'].dt.date == today_date.date())]
    
    # Count the number of events for today
    num_events_today = len(today_events)
    
    return num_events_today
```


### 2
Question: Do I have me with natalia tomorrow, True or False?
True answer: False
Non-executable code: Can only use .dt accessor with datetimelike values

```python
import pandas as pd

def query_dataframe(calendar_data):
    # get today's date
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # get tomorrow's date
    tomorrow_date = today_date + pd.DateOffset(days=1)
    
    # Filter the dataframe
    meetings_tomorrow = calendar_data[(calendar_data['start'].dt.date == tomorrow_date.date()) & 
                                       (calendar_data['attendees'].str.contains('natalia', case=False))]
    
    # Check if there are any meetings with Natalia tomorrow
    if len(meetings_tomorrow) > 0:
        return True
    else:
        return False
```


### 2 
Question: How many events are scheduled today?
 True answer: 2
 Non-executable code: Can only use .dt accessor with datetimelike values
 ```python
import pandas as pd

def query_dataframe(calendar_data):
    # get today's date
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # Filter the dataframe to only include events scheduled for today
    today_events = calendar_data[(calendar_data['start'].dt.date == today_date.date())]
    
    # Count the number of events scheduled for today
    num_events_today = len(today_events)
    
    return num_events_today
```


## Str and Timestamp

Question: What is the topic of my next meeting?
 True answer: Security Best Practices
 Non-executable code: '>=' not supported between instances of 'str' and 'Timestamp'

```python
def query_dataframe(calendar_data):
    
    import pandas as pd
    from datetime import datetime
    
    # get today's date
    today_date = pd.to_datetime('2024-04-02 09:02:30', format='%Y-%m-%d %H:%M:%S')
    
    # Filter the calendar_data to get the meetings happening after the current date and time
    upcoming_meetings = calendar_data[calendar_data['start'] >= today_date]
    
    # Sort the upcoming meetings based on start datetime
    upcoming_meetings = upcoming_meetings.sort_values('start')
    
    # Get the next meeting topic
    next_meeting_topic = upcoming_meetings.iloc[0]['summary']
    
    return next_meeting_topic
```

## This Next

## Last